In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [14]:
#chargements des caractéristiques
carac = pd.read_csv('carcteristiques-2022.csv', sep=';')
#conversion en chaine
cols_to_cast = ['Accident_Id', 'lum', 'agg', 'int', 'atm', 'col']
carac[cols_to_cast] = carac[cols_to_cast].astype('int')
#conversion des latitudes/longitudes
carac['lat']= carac['lat'].str.replace(',','.').astype('float')
carac['long']= carac['long'].str.replace(',','.').astype('float')

# suppression de l'adresse qui ne sert à rien
carac = carac.drop('adr', axis = 1)

#on ajoute un créneau horaire
carac['hh'] = carac['hrmn'].str[:2]

# traitement des -1 en NA
carac.replace({-1:np.nan}, inplace=True)

# gestion des NAs
# lum : 2 nas => valuers la plus fréquentes (1)
carac.loc[carac['lum'].isna(),'lum'] = 1

# int : 7 nas => valuers la plus fréquentes (1)
carac.loc[carac['int'].isna(),'int'] = 1

# atm : 1 nas => valuers la plus fréquentes (1)
carac.loc[carac['atm'].isna(),'atm'] = 1

# col : 63 nas => valuers la plus fréquentes (6) : atre collision
carac.loc[carac['col'].isna(),'col'] = 6

In [38]:
# chargement des usagers
usagers = pd.read_csv('usagers-2022.csv', sep=';')
#convesrion en chaine
cols_to_cast = ['Num_Acc', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'etatp']
usagers[cols_to_cast] = usagers[cols_to_cast].astype('int')

# suppression du code véhicucle alphanumérique redondant
usagers = usagers.drop('num_veh', axis = 1 )

#correction dees valeurs inconnue actp
usagers.loc[usagers['actp'].isin([' -1','8','9']),'actp'] = np.nan

# traitement des -1 en NA
usagers.replace({-1:np.nan}, inplace=True)

# place manquante : tous les usagers concernés sont conduxcteurs : donc on remplace par 1
usagers.loc[usagers['place'].isna(),'place'] = 1

# gravité : bcp d'information manquantes pour ces usagers : on supprime
usagers.dropna(subset=['grav'], inplace=True)

#sexe : le plus courant = H
usagers.loc[usagers['sexe'].isna()] = 1





In [41]:
usagers['trajet'].value_counts(dropna=False)

trajet
5.0    46253
0.0    32736
1.0    16282
4.0    11622
9.0    10494
3.0     3713
2.0     2687
NaN     2634
Name: count, dtype: int64

,Num_Acc,id_usager,id_vehicule,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,336537089,1 099 700,813 952,1.0,1,3.0,1.0,2008.0,5.0,2.0,8.0,NaN,NaN,NaN,NaN
1,336537089,1 099 701,813 953,1.0,1,1.0,1.0,1948.0,5.0,1.0,8.0,NaN,NaN,NaN,NaN
2,336537090,1 099 698,813 950,1.0,1,4.0,1.0,1988.0,9.0,1.0,0.0,NaN,0.0,0,NaN
3,336537090,1 099 699,813 951,1.0,1,1.0,1.0,1970.0,4.0,1.0,0.0,NaN,0.0,0,NaN
4,336537091,1 099 696,813 948,1.0,1,1.0,1.0,2002.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,336592389,968 230,715 631,1.0,1,1.0,2.0,2002.0,5.0,1.0,NaN,NaN,0.0,0,NaN
126658,336592389,968 231,715 631,8.0,2,3.0,2.0,2004.0,5.0,1.0,NaN,NaN,0.0,0,NaN
126659,336592389,968 232,715 632,1.0,1,4.0,2.0,1953.0,5.0,1.0,NaN,NaN,0.0,0,NaN
126660,336592390,968 228,715 629,1.0,1,3.0,1.0,1992.0,1.0,2.0,6.0,NaN,NaN,NaN,NaN


In [24]:
usagers.loc[usagers['grav'].isna()].isna().sum()

Num_Acc          0
id_usager        0
id_vehicule      0
place            0
catu             0
grav           241
sexe           240
an_nais        240
trajet         240
secu1          215
secu2          240
secu3          240
locp           225
actp           225
etatp          241
dtype: int64

In [35]:
usagers.loc[usagers['grav'].isna()].merge(carac, left_on='Num_Acc', right_on='Accident_Id').merge(usagers, left_on='Num_Acc', right_on='Num_Acc').groupby('Num_Acc').agg({'id_usager_y':'count'}).min()

id_usager_y    2
dtype: int64

In [33]:
carac.merge(usagers, how='left', left_on='Accident_Id', right_on='Num_Acc').groupby('Accident_Id').agg({'grav': 'min'}).value_counts().groupby('Num_Acc').agg({'id_usager_y':'count'})


KeyError: 'Num_Acc'

In [13]:
usagers.value_counts(dropna=False)

col
3.0    16838
6.0    16039
2.0     7419
1.0     5933
7.0     5531
4.0     1944
5.0     1535
NaN       63
Name: count, dtype: int64